In [18]:
import time
import pandas as pd
from plotly.subplots import make_subplots
import plotly.offline as offline
import plotly.graph_objs as graph_obj

start = time.time()


class StockGraphParser:

    __base_url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'

    @classmethod
    def initialize(cls):
        return

    @classmethod
    def __make_url(cls, code):
        return cls.__base_url.format(code=code)

    @classmethod
    def get_data_frame(cls, code):
        data_frame = pd.DataFrame()
        url = cls.__make_url(code)

        for page in range(1, 2):
            page_url = '{url}&page={page}'.format(url = url, page = page)
            print(page_url)
            data_frame = data_frame.append(pd.read_html(page_url, header = 0)[0], ignore_index = True)

        data_frame = data_frame.dropna()
        
        # TODO : Remove duplicated (check by date)
        return data_frame
    
    
data_frame = StockGraphParser.get_data_frame('002700')

# Show raw data
data_frame

# Rename columns
data_frame = data_frame.rename(columns= {'날짜': 'date', '종가': 'close', '전일비': 'diff', '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'})

# Convert data type
data_frame[['close', 'diff', 'open', 'high', 'low', 'volume']] = data_frame[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int)

data_frame['date'] = pd.to_datetime(data_frame['date'])

# Sort
data_frame = data_frame.sort_values(by = ['date'], ascending=True)


item_name = '신일전자'

# GRAPH PART
# jupyter notebook 에서 출력 
offline.init_notebook_mode(connected=True)

fig = make_subplots(rows=2, cols=1, shared_xaxes = True)

fig.add_trace(graph_obj.Candlestick(
                          x=data_frame.date,
                          open = data_frame.open,
                          high = data_frame.high,
                          low = data_frame.low,
                          close = data_frame.close,
                          # y=data_frame.close,
                          name=item_name
), row=1, col=1)

fig.add_trace(graph_obj.Bar(x = data_frame.date,
                            y = data_frame.volume,
                            name = '거래량')
                            
), row=2, col=1)

fig.update_layout('rangeslider' : {
                    'visible' : False
})
fig.show()

print("Get - Show :", time.time() - start) 




SyntaxError: invalid syntax (<ipython-input-18-4df934f6a7ff>, line 78)